In [1]:
import numpy as np
import scipy
import utils
from utils import HouseHolder, QR, SVD

%load_ext autoreload
%autoreload 2

# Problem 1: SVD by Two-Phase Approach

## Phase-I: Golub-Kahan Bidiagonalization

In [4]:
A = np.array([[0, 0, 0, 0],
              [0, 0, 0, 0],
              [0, 0, 1, 0],
              [0, 0, 0, 0],
              [2, 5, 0, 0],
              [0, 0, 0, 0],
              [0, 0, 0, 0]], dtype=np.float64).T


# A = np.array([[1, 0, 1],
#               [2, 5**.5, 0],
#               [0, 0, 1],
#               [0, 0, 1]])
B, Qt, P = SVD.svd_phaseI(A)
print(B)


[[ 2.  0.  0.  0.  0.  0.  0.]
 [-5.  0.  0.  0.  0.  0.  0.]
 [ 0.  0. -1.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]]


In [5]:
print(Qt @ A @ P)

[[ 2.  0.  0.  0.  0.  0.  0.]
 [-5.  0.  0.  0.  0.  0.  0.]
 [ 0.  0. -1.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]]


Todo: We may not need to form the reflection matrix H explicitly during Golub-Kahan bidiagonalization.

## Phase-II-A

**Test eigen solver**

In [2]:
%precision 4

'%.4f'

In [17]:
m = n = 500
A = np.random.rand(m,n)
A, _, _ = SVD.svd_phaseI(A)
A = A@A.T


In [18]:
T, Q = QR.tmp_eigh(A, maxn=10)
print(np.sum(np.abs(Q@np.diag(T)@Q.T - A) < 1e-8))

250000


In [19]:
T, Q = QR.eigh_by_QR(A, maxn=10)
print(np.sum(np.abs(Q@np.diag(T)@Q.T - A) < 1e-8))

250000


**Test SVD**

In [22]:
m = n = 1000
A = np.random.rand(m,n)

In [23]:
U, S, Vt = SVD.svd(A)
# print(U@np.diag(S)@Vt)
# print(np.abs(U@np.diag(S)@Vt - A))
print(np.sum(np.abs(U@np.diag(S)@Vt - A)< 1e-13))
U, S, Vt

phaseI: 45.054675579071045
phaseII: 79.85789656639099
1000000


(array([[-0.0313, -0.0326,  0.0121, ...,  0.0031, -0.0362,  0.069 ],
        [-0.0308, -0.0054, -0.0077, ..., -0.0052,  0.0463,  0.0034],
        [-0.0303,  0.03  ,  0.0133, ...,  0.007 , -0.0088,  0.0518],
        ...,
        [-0.0318,  0.0039, -0.0062, ..., -0.0082, -0.0136, -0.034 ],
        [-0.0325,  0.0226,  0.0199, ...,  0.0059, -0.0142, -0.0406],
        [-0.0318, -0.0311, -0.0078, ...,  0.0554, -0.0177,  0.005 ]]),
 array([5.0045e+02, 1.8129e+01, 1.7992e+01, 1.7873e+01, 1.7832e+01,
        1.7761e+01, 1.7672e+01, 1.7626e+01, 1.7618e+01, 1.7556e+01,
        1.7480e+01, 1.7475e+01, 1.7387e+01, 1.7337e+01, 1.7319e+01,
        1.7227e+01, 1.7188e+01, 1.7171e+01, 1.7138e+01, 1.7111e+01,
        1.7083e+01, 1.7039e+01, 1.6992e+01, 1.6935e+01, 1.6907e+01,
        1.6897e+01, 1.6821e+01, 1.6789e+01, 1.6751e+01, 1.6725e+01,
        1.6710e+01, 1.6637e+01, 1.6600e+01, 1.6572e+01, 1.6553e+01,
        1.6528e+01, 1.6489e+01, 1.6436e+01, 1.6428e+01, 1.6401e+01,
        1.6367e+01, 1.6337e

15.1-9.7 <br>
15.5-10.5

In [9]:
U, S, Vt  = scipy.linalg.svd(A, full_matrices=False)
# print(np.abs(U@np.diag(S)@Vt - A))
print(np.sum(np.abs(U@np.diag(S)@Vt - A)< 1e-13))
U, S, Vt

1048576


(array([[-0.031 , -0.0079, -0.031 , ...,  0.0236, -0.0389,  0.0148],
        [-0.0307,  0.0149, -0.0027, ...,  0.0223, -0.075 , -0.0087],
        [-0.0299,  0.018 ,  0.0271, ..., -0.0546,  0.0488,  0.0285],
        ...,
        [-0.0307,  0.0122, -0.0187, ...,  0.0494,  0.0392, -0.0157],
        [-0.032 ,  0.0215,  0.0264, ..., -0.0033, -0.0158,  0.0102],
        [-0.0307, -0.0016,  0.0182, ...,  0.0572,  0.0323, -0.0378]]),
 array([5.1205e+02, 1.8386e+01, 1.8278e+01, ..., 1.8997e-02, 1.0529e-02,
        1.6051e-03]),
 array([[-0.0306, -0.0313, -0.0306, ..., -0.0316, -0.0316, -0.0309],
        [-0.0113, -0.015 , -0.0339, ...,  0.0244,  0.0364,  0.0505],
        [ 0.0049, -0.0784,  0.0272, ...,  0.0052, -0.0015, -0.0652],
        ...,
        [ 0.0169,  0.0217, -0.0281, ..., -0.0257, -0.0362,  0.0261],
        [ 0.0334, -0.0018, -0.0285, ..., -0.0341, -0.0225,  0.0613],
        [ 0.0437,  0.0073, -0.0002, ...,  0.0267, -0.0071,  0.0326]]))